# Advent of Code 2022 - Day 9

In [1]:
from pathlib import Path
from typing import Final, List, Dict, Tuple, Set, Any, Optional

## Part One

- [X] Read input
- [ ] Split each instruction to a Tuple of (Direction, Number)
- [ ] Create a grid for the rope to move on?
- [ ] Move Head
- [ ] Compare Tail position to head and move accordingly
- [ ] Mark position of tail on a boolean grid (True is tail has passed that space)
- [ ] Count grid


In [2]:
INPUT_NAME : str = "day9_input_test.txt"
# MovePosition : TypeAlias = Tuple[int, int]

In [3]:
def read_input(input_name: str) -> List[str]:
    input_path : Path = Path.cwd() / "input" / input_name

    with open(input_path, "r") as input_file:

        lines : List = input_file.readlines()

    return [line.rstrip("\n") for line in lines]

In [4]:
instructions_input : List= [tuple(line.split(" ")) for line in read_input(INPUT_NAME)]

In [5]:

STARTING_POSITION : Final= (0,0)
STEP = 1
ROW = 0
COL = 1
DIAGONAL = "/"
DIRECTION = 0
COUNT = 1

In [6]:
all_covered_positions = set([])

In [7]:
def make_step_move(current_position, move, is_tail: bool = False):
    """
    Co-ordinates are relative to start (0,0), as (row, col)
    """  
    
    return (current_position[ROW] + move[ROW], current_position[COL] + move[COL])
    

In [8]:
def move_head(current_position, move):
    
    return make_step_move(current_position, move)

In [9]:
def move_tail(direction: str, head_position, current_position):
    """
    Delta position: 
        -ve means below or to the left of head
    
    Diagonal movement:
        Head movmement that results in a diagonal delta should result in no tail movement.
        Next tail movement should then be to where head was in previous.
    """
    
    # delta_position = (current_position[ROW]-head_position[ROW], current_position[COL]-head_position[COL])
    delta_position = (head_position[ROW]-current_position[ROW], head_position[COL]-current_position[COL])
    # print(f"Delta: {delta_position}")
    
    # Check if in same row/col (else diagonal move)
    # Both checks will be false if diagonal move required
    if max(abs(delta_position[ROW]), abs(delta_position[COL])) < 2: # Head on top of tail! (would only be if U then D, or L then R)
        # print("Returning current")
        return current_position
    
    elif current_position[ROW] == head_position[ROW] or current_position[COL] == head_position[COL]:
        # Move orthogonally
        move = generate_step_move(direction)
        # print("Orthogonal")
        return make_step_move(current_position, move)
    else:
        # Move diagonally
        if max(abs(delta_position[ROW]), abs(delta_position[COL])) <= 1:
        # If delta position is (1,1) then don't move
            # print("Diagonal, no move")
            return current_position

        else:
        # If |delta position| is (2,1) or (1,2) then move diagonally
            # print("Diagonal, move")
            move = generate_step_move(DIAGONAL, delta_position)
            return make_step_move(current_position, move)

In [10]:
def generate_step_move(direction: str, diagonal_delta = None):
    
    step = -1 if direction in ["L", "D"] else 1
    
    if direction in ["L", "R"]:
        move = (0, step)
    elif direction in ["U", "D"]:
        move = (step, 0)
    elif direction == DIAGONAL:
        
        def check_diag_move(num_spaces: int) -> int:
            """
                If a row or col move is > 2 it is diagonal
                Function converts this to a tail move
            """
            
            if abs(num_spaces) > 1:
                if 1-(num_spaces<0):    # num_spaces is positive
                    return num_spaces -1
                else:                   # num_spaces is negative
                    return num_spaces + 1
            else:
                return num_spaces
            
        move = (check_diag_move(diagonal_delta[ROW]), check_diag_move(diagonal_delta[COL]))
        # print(f"MOVE: {move}")
        
    return move

In [11]:
def make_move(
    direction: str, count: str, head_position, tail_position
):
    """
    Returns current head position, and list of positions tail has been in for this move
    """

    count = int(count)

    current_head_position = head_position
    current_tail_position = tail_position
    covered_positions : List = []

    for idx in range(count):
        # print(f"{idx}")
        start_pos : bool = current_head_position == STARTING_POSITION
        move = generate_step_move(direction)
        current_head_position = move_head(current_head_position, move)
        if not start_pos:
            # print("Moving tail!")
            current_tail_position = move_tail(direction, current_head_position, current_tail_position)
            covered_positions.append(current_tail_position)
        else:
            covered_positions.append(STARTING_POSITION)
        # print(f"Head: {current_head_position}, Tail: {current_tail_position}")
        
    return current_head_position, current_tail_position, covered_positions
        

In [12]:
# position_set.update(make_move("U", 4, STARTING_POSITION, STARTING_POSITION)[2])

In [13]:
# Initialise a list of starting positiong
# Indexing is 0 = Head, 1-9 = Tails
knots = [STARTING_POSITION] * 10

for idx, instruction in enumerate(instructions_input):

    # if idx == 0:
    #     current_head_position, current_tail_position = (
    #         STARTING_POSITION,
    #         STARTING_POSITION,
    #     )
        
    # print(f"[{idx}] {instruction[DIRECTION]}, {instruction[COUNT]}")

    for head_idx, knot in enumerate(knots):

        tail_idx = head_idx + 1
        if tail_idx < len(knots):
            
            current_head_position = knots[head_idx]
            current_tail_position = knots[tail_idx]
        
            knots[head_idx], knots[tail_idx], covered_positions = make_move(
                instruction[DIRECTION],
                instruction[COUNT],
                current_head_position,
                current_tail_position,
            )
            
        if tail_idx == len(knots) -1:
            all_covered_positions.update(covered_positions)
    
    
print(len(all_covered_positions))

22


In [14]:
# def follow_instruction(instruction: Tuple[str, str], current_head_position, current_tail_position):

#     current_head_position, current_tail_position, covered_positions = make_move(
#         instruction[DIRECTION],
#         instruction[COUNT],
#         current_head_position,
#         current_tail_position,
#     # 

#     all_covered_positions.update(covered_positions)
    
#     return current_head_position, current_tail_position

In [15]:
# # Starting Pos
# move1_r4 = follow_instruction(instructions_input[0], STARTING_POSITION, STARTING_POSITION)

## Part Two

"Should" be able to update the code to simply use a list of Tail positions, each using the tail in front as it's head

In [16]:
# Initialise a list of starting positiong